In [2]:
import yfinance as yf
import pandas as pd 
import numpy as np 


In [20]:
ticker = 'SPY'
data = yf.download(ticker, start='2010-01-01', end='2025-06-30')

C:\Users\nickt\AppData\Local\Temp\ipykernel_17016\65919978.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2010-01-01', end='2025-06-30')
[*********************100%***********************]  1 of 1 completed


In [ ]:
# Flatten MultiIndex columns
data.columns = data.columns.get_level_values(0)

# Reset index to turn 'Date' from index to a column
data = data.reset_index()

# Rename columns
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']


In [25]:
data

,Date,Open,High,Low,Close,Volume
0,2010-01-04,85.515640,85.560913,84.142320,84.791252,118944600
1,2010-01-05,85.741982,85.779712,85.153416,85.462794,111579900
2,2010-01-06,85.802361,86.013640,85.591081,85.658990,116074400
3,2010-01-07,86.164558,86.270198,85.402439,85.643902,131091100
4,2010-01-08,86.451279,86.489010,85.764621,85.938170,126402800
...,...,...,...,...,...,...
3890,2025-06-23,600.150024,600.539978,591.890015,595.039978,87426000
3891,2025-06-24,606.780029,607.849976,603.409973,604.330017,67735300
3892,2025-06-25,607.119995,608.609985,605.539978,607.909973,62114800
3893,2025-06-26,611.869995,612.309998,608.369995,608.989990,78548400


In [28]:
data['dailyret'] = data['Close'].pct_change()
data['state'] = np.where(data['dailyret'] >= 0, "up", "down")

In [29]:
data

,Date,Open,High,Low,Close,Volume,dailyret,state
0,2010-01-04,85.515640,85.560913,84.142320,84.791252,118944600,NaN,down
1,2010-01-05,85.741982,85.779712,85.153416,85.462794,111579900,0.007920,up
2,2010-01-06,85.802361,86.013640,85.591081,85.658990,116074400,0.002296,up
3,2010-01-07,86.164558,86.270198,85.402439,85.643902,131091100,-0.000176,down
4,2010-01-08,86.451279,86.489010,85.764621,85.938170,126402800,0.003436,up
...,...,...,...,...,...,...,...,...
3890,2025-06-23,600.150024,600.539978,591.890015,595.039978,87426000,-0.005582,down
3891,2025-06-24,606.780029,607.849976,603.409973,604.330017,67735300,0.015612,up
3892,2025-06-25,607.119995,608.609985,605.539978,607.909973,62114800,0.005924,up
3893,2025-06-26,611.869995,612.309998,608.369995,608.989990,78548400,0.001777,up


In [52]:
# Probablity of daily return being up or down given the previous day's state

up_counts = len(data[data['state'] == 'up'])
down_counts = len(data[data['state'] == 'down'])

previous_state = data['state'].shift(1)

up_to_up = ((previous_state == 'up') & (data['state'] == 'up')).sum() / up_counts
up_to_down = ((previous_state == 'up') & (data['state'] == 'down')).sum() / up_counts
down_to_up = ((previous_state == 'down') & (data['state'] == 'up')).sum() / down_counts
down_to_down = ((previous_state == 'down') & (data['state'] == 'down')).sum() / down_counts

transition_matrix = pd.DataFrame({
    'up': [up_to_up, down_to_up],
    'down': [up_to_down, down_to_down]
}, index=['up', 'down'])

print(transition_matrix)

            up      down
up    0.560037  0.439505
down  0.560420  0.439580


In [79]:
# Probability of negative daily return after being in a uptrend for x days

x = 1
consecutive_up = (data['state'] == 'up').astype(int).rolling(x).sum() == x
next_state = data['state'].shift(-1)
down_after_consecutive_up = len(data[consecutive_up & (next_state == 'down')])
print(down_after_consecutive_up)
print(consecutive_up.sum())
print(down_after_consecutive_up / consecutive_up.sum())

959
2182
0.4395050412465628


In [70]:
# Probability of positive daily return after being in a downtrend for x days

x = 6
consecutive_down = (data['state'] == 'down').astype(int).rolling(x).sum() == x
next_state = data['state'].shift(-1)
up_after_consecutive_down = len(data[consecutive_down & (next_state == 'up')])
print(up_after_consecutive_down)
print(consecutive_down.sum())
print(up_after_consecutive_down/ consecutive_down.sum())

18
30
0.6
